In [ ]:
import threading
import time
import gensim.downloader as api
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
import os
from gensim.models.keyedvectors import KeyedVectors
from PIL import Image, ImageTk
import re
import jellyfish
from typing import List, Set
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModelForSequenceClassification, AdamW
import torch
from bs4 import BeautifulSoup
from fuzzywuzzy import fuzz

In [ ]:
stop_event = threading.Event()
# Kiểu mô hình mặc định
default_model = "PhoBERT"
model = None
tokenizer = None
word2vec_model = None
doc2vec_model = None
test1_path = None
test2_path = None
# độ dài mã thông báo
MAX_LENGTH = 512
local_model_dir = "./models"
matrix_file_path1 = None
matrix_file_path2 = None

In [ ]:
file_path = 'vietnamese-stopwords.txt'
with open(file_path, 'r', encoding='utf-8') as file:
    STOP_WORDS = set(file.read().splitlines())

print(STOP_WORDS)

In [ ]:
def remove_stopwords(text, stopwords):
    words = text.split()
    filtered_words = [word for word in words if word not in stopwords]
    return ' '.join(filtered_words)

In [ ]:
df = pd.read_csv('D:/Python/Quora/Quora/quora-question-pairs/Data/Dataset/train/dataset.csv')

In [ ]:
new_df = df

In [ ]:
new_df.head()

In [ ]:
def preprocess(q):
    
    q = str(q).lower().strip()
    
    # Replace certain special characters with their string equivalents
    q = q.replace('%', ' percent')
    q = q.replace('$', ' dollar ')
    q = q.replace('₹', ' rupee ')
    q = q.replace('€', ' euro ')
    q = q.replace('@', ' at ')
    
    # The pattern '[math]' appears around 900 times in the whole dataset.
    q = q.replace('[math]', '')
    
    # Replacing some numbers with string equivalents (not perfect, can be done better to account for more cases)
    q = q.replace(',000,000,000 ', 'b ')
    q = q.replace(',000,000 ', 'm ')
    q = q.replace(',000 ', 'k ')
    q = re.sub(r'([0-9]+)000000000', r'\1b', q)
    q = re.sub(r'([0-9]+)000000', r'\1m', q)
    q = re.sub(r'([0-9]+)000', r'\1k', q)
    
    contractions = { 
    "sp": "sư phạm",
    "sn": "sinh năm",
    "bn": "bao nhiêu",
    "ngta": "người ta",
    "lm": "làm",
    "khg": "không",
    "ko": "không",
    "hok": "không",
    "khum": "không",
    "kg": "không",
    "đc": "được",
    "dc": "được",
    "đg": "đang",
    "ng": "người",
    "bt": "biết",
    "h": "giờ",
    "hc": "học",
    "vt": "viết",
    "vc": "việc",
    "trc": "trước",
    "j": "gì",
    "xg": "xong",
    "bx": "bữa",
    "vg": "vâng",
    "v": "vậy",
    "m": "mày",
    "t": "tôi",
    "s": "sao",
    "r": "rồi",
    "chs": "chơi",
    "ae": "anh em",
    "a": "anh",
    "e": "em",
    "cj": "chị",
    "đhsp": "đại học sư phạm",
    "đh": "đại học",
    "đhspkt": "đại học sư phạm kỹ thuật",
    "ktx": "kí túc xá",
    "đt": "điện thoại",
    "cntt": "công nghệ thông tin",
    "tp": "thành phố",
    "hcm": "hồ chí minh",
    "ain't": "am not",
    "aren't": "are not",
    "can't": "can not",
    "can't've": "can not have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "i'd": "i would",
    "i'd've": "i would have",
    "i'll": "i will",
    "i'll've": "i will have",
    "i'm": "i am",
    "i've": "i have",
    "isn't": "is not",
    "it'd": "it would",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so as",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there would",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you would",
    "you'd've": "you would have",
    "you'll": "you will",
    "you'll've": "you will have",
    "you're": "you are",
    "you've": "you have"
    }

    q_decontracted = []

    for word in q.split():
        if word in contractions:
            word = contractions[word]

        q_decontracted.append(word)

    q = ' '.join(q_decontracted)
    q = q.replace("'ve", " have")
    q = q.replace("n't", " not")
    q = q.replace("'re", " are")
    q = q.replace("'ll", " will")
    
    # Removing HTML tags
    q = BeautifulSoup(q)
    q = q.get_text()
    # Remove punctuations
    pattern = re.compile('\W')
    q = re.sub(pattern, ' ', q).strip()

    
    return q

In [ ]:
new_df['question1'] = new_df['question1'].apply(preprocess)
new_df['question2'] = new_df['question2'].apply(preprocess)

In [ ]:
new_df.head()

In [ ]:
def ensure_dir_exists(dir_path):
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

In [ ]:
ensure_dir_exists(local_model_dir)

In [ ]:
def load_or_download_phobert():
    phobert_model_path = os.path.join(local_model_dir, "models--vinai--phobert-base")
    if not os.path.exists(phobert_model_path):
        print("Downloading PhoBERT model...")
        AutoModel.from_pretrained("vinai/phobert-base", cache_dir=local_model_dir)
        AutoTokenizer.from_pretrained("vinai/phobert-base", cache_dir=local_model_dir)
    else:
        print("PhoBERT model loaded from local storage.")
    return AutoModel.from_pretrained(
        "vinai/phobert-base", cache_dir=local_model_dir
    ), AutoTokenizer.from_pretrained("vinai/phobert-base", cache_dir=local_model_dir)

In [ ]:
def preload_models():
    global model, tokenizer
    try:
        if not os.path.exists(local_model_dir):
            os.makedirs(local_model_dir)
            print(f"Directory created: {local_model_dir}")
        ensure_dir_exists(local_model_dir)
        model, tokenizer = load_or_download_phobert()
        print(f"Models loaded successfully.")
        time.sleep(1)
    except Exception as e:
        print(f"Error loading models: {str(e)}")

In [ ]:
#Chuẩn hóa ma trận tương đồng về khoảng [0, 1].
def normalize_similarity_matrix(matrix):
    return (matrix + 1) / 2

In [ ]:
#Trích xuất embedding của câu bằng PhoBERT.
def get_embedding(sentence):
    inputs = tokenizer(
        sentence, 
        return_tensors="pt", 
        truncation=True, 
        padding=True, 
        max_length=128
    )
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().detach().numpy()

In [ ]:
#Tính độ tương đồng cosine giữa hai câu.
def sentence_similarity(sent1, sent2):
    embed1 = get_embedding(sent1)
    embed2 = get_embedding(sent2)
    cosine_similarity = np.dot(embed1, embed2) / (
        np.linalg.norm(embed1) * np.linalg.norm(embed2)
    )
    return cosine_similarity

In [ ]:
#Tính độ tương đồng sử dụng PhoBERT.
def calculate_similarity_phobert(questions):
    try:
        normalized_similarity_matrix = sentence_similarity(questions[0], questions[1])
        return normalized_similarity_matrix
    except Exception as e:

        raise RuntimeError(f"Error calculate similarity Phobert: {str(e)}")

In [ ]:
#Gọi hàm tính toán độ tương đồng tùy thuộc vào loại mô hình.
def calculate_similarity(questions, model_type):
    questions1 = questions.copy()
    if model_type == "PhoBERT":
        return calculate_similarity_phobert(questions1)

In [ ]:
# Khai báo biến toàn cục
data_file_path1 = ""
data_file_path2 = ""
keywords_file_path = ""
matrix_tab3 = []

In [ ]:
# Hàm đọc từ khóa từ file Excel
def load_keywords_from_excel(
    file_path: str, sheet_name: str, column_name: str
) -> Set[str]:
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    keywords = set(df[column_name].dropna().str.strip().str.lower())
    return keywords

In [ ]:
# Hàm kiểm tra từ khóa dựa trên ngưỡng tương đồng Jaro-Winkler
def is_keyword_present_with_threshold(
    keyword: str, words: List[str], threshold: float = 0.6
) -> int:
    for word in words:
        similarity = jellyfish.jaro_winkler_similarity(keyword, word)
        if similarity >= threshold:
            return 1
    return 0

In [ ]:
# Hàm tạo véc-tơ đặc trưng cho câu hỏi
def create_feature_vector(
    words: List[str], keywords: Set[str], threshold: float = 0.6
) -> List[int]:
    feature_vector = [
        is_keyword_present_with_threshold(keyword, words, threshold)
        for keyword in keywords
    ]
    return feature_vector

In [ ]:
# Hàm tính độ tương đồng Jaccard
def jaccard_similarity(vec1: List[int], vec2: List[int]) -> float:
    intersection = sum(1 for v1, v2 in zip(vec1, vec2) if v1 == 1 and v2 == 1)
    union = sum(1 for v1, v2 in zip(vec1, vec2) if v1 == 1 or v2 == 1)
    return intersection / union if union != 0 else 0

In [ ]:
preload_models()

In [ ]:
train_df, test_df = train_test_split(new_df, test_size=0.2, random_state=42)

In [ ]:
def preprocess_data(new_df):
    inputs = tokenizer(
        new_df['question1'].tolist(),
        new_df['question2'].tolist(),
        truncation=True,
        padding=True,
        max_length=128,
        return_tensors="pt"
    )
    inputs['labels'] = new_df['is_duplicate'].tolist()
    return inputs

In [ ]:
# Áp dụng tiền xử lý
train_data = preprocess_data(train_df)
test_data = preprocess_data(test_df)

In [ ]:
class QuestionPairDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
# Tạo Dataset
train_dataset = QuestionPairDataset(train_data, train_df['is_duplicate'].tolist())
test_dataset = QuestionPairDataset(test_data, test_df['is_duplicate'].tolist())

In [ ]:
# Tạo DataLoader
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [ ]:
# Load mô hình PhoBERT với đầu ra là phân loại nhị phân
model = AutoModelForSequenceClassification.from_pretrained("vinai/phobert-base", num_labels=2)
model_embed = AutoModel.from_pretrained("vinai/phobert-base")
# Thiết lập optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)

In [ ]:
if torch.cuda.is_available():
    print("GPU is available and being used.")
else:
    print("GPU is not available, using CPU instead.")

In [ ]:
# Huấn luyện mô hình
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(5):  # Số epoch
    model.train()
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1}, Loss: {total_loss / len(train_loader)}")

In [ ]:
#đánh giá mô hình
from sklearn.metrics import accuracy_score

model.eval()
predictions, true_labels = [], []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1).cpu().numpy()

        predictions.extend(preds)
        true_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(true_labels, predictions)
print(f"Accuracy: {accuracy:.4f}")

In [ ]:
model.save_pretrained("./phobert-finetuned")
tokenizer.save_pretrained("./phobert-finetuned")
model_embed.save_pretrained("./phobert-embed")

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import faiss

# Tải lại mô hình và tokenizer
model = AutoModelForSequenceClassification.from_pretrained("./phobert-finetuned")
tokenizer = AutoTokenizer.from_pretrained("./phobert-finetuned")
model_embed = AutoModel.from_pretrained("./phobert-embed")
# Load FAISS index
index = faiss.read_index("faiss_index.bin")

# Dự đoán độ tương đồng
def predict_similarity(question1, question2):
    inputs = tokenizer(question1, question2, return_tensors="pt", truncation=True, padding=True, max_length=128)
    outputs = model(**inputs)
    probs = torch.softmax(outputs.logits, dim=1)
    return probs[0][1].item()  # Xác suất thuộc lớp tương đồng (1)

In [ ]:
def find_answer(input_question, dataset, threshold=0.6):
    # Tính độ tương đồng giữa câu hỏi đầu vào và các câu hỏi trong bộ dữ liệu
    max_similarity = -1
    best_match = None

    for index, row in dataset.iterrows():
        # Tính độ tương đồng giữa input_question và question1
        similarity1 = predict_similarity(input_question, row['question1'])
        # Tính độ tương đồng giữa input_question và question2
        similarity2 = predict_similarity(input_question, row['question2'])
        # Lấy độ tương đồng cao nhất
        max_current_similarity = max(similarity1, similarity2)

        # Nếu độ tương đồng cao hơn ngưỡng và cao hơn giá trị hiện tại
        if max_current_similarity > threshold and max_current_similarity > max_similarity:
            max_similarity = max_current_similarity
            best_match = row

    # Nếu tìm thấy câu hỏi trùng khớp, trả về câu trả lời
    if best_match is not None:
        return best_match['answer']
    else:
        return "Không tìm thấy câu trả lời phù hợp."

In [ ]:
import faiss
def encode(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=256)
    with torch.no_grad():
        outputs = model_embed(**inputs)
    return outputs.last_hidden_state[:, 0, :].squeeze(0).cpu().numpy()  # Chuyển về 1D

# Tạo danh sách vector từ các câu hỏi
question_vectors = np.array([encode(row['question1']) for _, row in df.iterrows()])

# Kiểm tra kích thước
print(question_vectors.shape)  # Kết quả phải là (số lượng câu, 768)

# Khởi tạo FAISS Index
index = faiss.IndexFlatL2(question_vectors.shape[1])
index.add(question_vectors)  # Không còn lỗi

In [ ]:
# Lưu FAISS index
faiss.write_index(index, "faiss_index.bin")

# Lưu các câu hỏi dưới dạng numpy để khôi phục sau này
np.save("questions.npy", df[['question1', 'question2', 'answer']].to_numpy(), allow_pickle=True)

In [ ]:

def find_answer_faiss(input_question, df):
    input_vector = encode(input_question).astype('float32')
    input_vector = input_vector.reshape(1, -1)
    D, I = index.search(input_vector, 1)  # Tìm 1 câu gần nhất
    best_match = df.iloc[I[0][0]]
    return best_match['answer'] if D[0][0] < 0.6 else "Không tìm thấy câu trả lời phù hợp."

In [ ]:
# Ví dụ
question1 = "Trang Facebook nào nhất ở Ấn Độ?"
question2 = "Khi nào Facebook ra mắt ở Ấn Độ?"

In [ ]:
test_df = new_df.head(5)
test_df['similarity_score'] = test_df.apply(lambda row: predict_similarity(row['question1'], row['question2']), axis=1)
test_df['answer_question'] = test_df.apply(lambda row: find_answer_faiss(row['question1'], df), axis=1)
test_df

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(true_labels, predictions))